# Who: Andrei Olariu
## Stone Soup Technology
<br/>
<br/>


# What: Matching Journalists with Domain Experts
## Text Classification with BERT and Gradient Boosting Trees from Idea to Production

- intro on me and StoneSoup

- describing the client and his problem

- baseline approach

- Machine Learning approach

- discussion

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/sst.png" width="800vmin" style="padding: 0 0 0 0">
  </center>
</figure>

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/andrei.jpg" width="1200vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

that's me during the pandemic

phd in natural language processing

3 top 10 finishes in kaggle contests

at sst, involved in backend, api and ml work

now you have more context on me and the company I work at

unfortunately, I don't know a lot about you, so I had to prepare this workshop making some assumptions

if you have experience with something and you find some things trivial, then just breath in, breath out, enjoy the moment and bask in your awesomeness

don't wait till the end for questions

# Assumptions

- no/little knowledge of Python
- no/little experience building and deploying Machine Learning models

# Feel free to interrupt me and ask questions

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/guidedpr.png" width="1200vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

We have a database of members, with a short description for each

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/brad.png" width="1200vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

We have new requests coming in from journalists and we want to identify the best matching members

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/request1.png" width="1200vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

What do you think?

How can we approach this problem?

We need a baseline approach. It needs to be:

- easy to implement

- easy to explain/understand/debug

- predictable

- independent of training data

Cmon, let's hear some ideas

actively thinking about a solution will stimulate the creation of new neural paths inside your brain, making you better problem solvers

i swear I'm not moving forward until I get another idea

Our baseline approach:

- index the members' descriptions using a search engine (Solr)

- given a request, send it to the search engine as the query and get the highest ranking members

- **have a moderator review and correct the matches**

- send updated matches to the journalist

notice the third step (hard not to)

we don't trust the current approach - and it's right so, we haven't evaluated it

we are also building a manually annotated dataset to train a ML model

In [21]:
import pickle

with open('matches.pickle', 'rb') as f:
    matches = pickle.loads(f.read())

In [22]:
len(matches)

39375

In [23]:
matches[0]

{'member_id': 113,
 'request_id': 28552,
 'mismatch': False,
 'auto_generated': True}

In [24]:
matches[-1]

{'member_id': 'Andrei',
 'request_id': 'Codiax',
 'mismatch': True,
 'auto_generated': False,
 'message': 'remove this from the matches list'}

In [25]:
matches = matches[:-1]

In [27]:
y = [] # the correct labels
baseline_predictions = []

# write code here...


# .. so that the asserts pass
assert len(y) == len(baseline_predictions) == len(matches)
assert sum(y) == 15739
assert sum(baseline_predictions) == 37010

In [28]:
y = [] # the correct labels
baseline_predictions = []

# write code here...
for match in matches:
    if match['auto_generated']:
        baseline_predictions.append(1)
        if match['mismatch']:
            y.append(0)
        else:
            y.append(1)
    else:
        baseline_predictions.append(0)
        if match['mismatch']:
            raise Exception('this is not possible', match)
        else:
            y.append(1)

# .. so that the asserts pass
assert len(y) == len(baseline_predictions) == len(matches)
assert sum(y) == 15739
assert sum(baseline_predictions) == 37010

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/acc.png" width="700vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

In [29]:
import numpy as np

y = np.array(y)
baseline_predictions = np.array(baseline_predictions)

sum(baseline_predictions == y) / len(y)

0.3396911667597907

is this a good score or a bad score?

if doing pedestrian detection for autonomous driving, probably a bad score

but if you search for something on google and 3 of the first 10 results are what you were looking for, then it's probably a good score

in our case, we can ask for feedback from the moderators, since they're the ones that see these results

apart from that, for us this is just a number out of context

speaking of context, does anybody feel we might be missing something here?

this formula for accuracy is the general one; for binary classification we have the equivalent, but slightly more detailed:

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/acc2.png" width="700vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/confusion-matrix.png" width="800vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

did anybody figure out where i'm going with this?

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/acc2.png" width="700vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

<figure style="display: table; margin: 0 auto">
  <center>
    <img src="images/confusion-matrix2.png" width="800vmin" style="padding: 4vmin 0 0 0">
  </center>
</figure>

so it seems like we have no true negatives in our dataset

let's say we have 100 members and the algorithm only returns 5 matches; let's say the moderator adds another 5 members to the results; the remaining 90 members are the true negatives; both the algorithm and the human moderator haven't selected them; but they are not in our dataset

we need to add true negative to our dataset:
- for a better understanding of our algorithm's performance
- for training a ML model



In [ ]:
true_negatives = []